<a href="https://colab.research.google.com/github/bhas10bc/Deep_learning/blob/main/kaggle_defects_binary_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [421]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [422]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

In [423]:
test_data

,id,loc,v(g),ev(g),iv(g),n,v,l,d,i,...,t,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount
0,101763,33.0,5.0,1.0,4.0,144.0,824.82,0.04,26.96,30.05,...,1257.60,30,0,3,0,21.0,23.0,87.0,57.0,9.0
1,101764,27.0,8.0,8.0,2.0,125.0,646.24,0.04,22.82,27.22,...,804.58,23,0,2,0,18.0,19.0,70.0,49.0,15.0
2,101765,130.0,11.0,7.0,10.0,545.0,3831.40,0.02,48.15,66.17,...,6453.34,99,9,17,1,26.0,53.0,333.0,244.0,21.0
3,101766,65.0,7.0,1.0,7.0,156.0,855.71,0.06,17.23,49.89,...,896.42,45,8,10,0,15.0,26.0,88.0,60.0,13.0
4,101767,22.0,3.0,1.0,3.0,52.0,238.42,0.10,9.60,26.70,...,145.80,16,0,4,0,12.0,15.0,30.0,24.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67837,169600,41.0,1.0,1.0,1.0,125.0,656.55,0.07,14.00,47.61,...,479.35,31,0,7,0,14.0,27.0,71.0,54.0,1.0
67838,169601,20.0,3.0,1.0,3.0,38.0,161.42,0.15,6.75,23.28,...,58.94,15,0,3,0,9.0,10.0,23.0,15.0,5.0
67839,169602,24.0,2.0,1.0,2.0,52.0,240.00,0.11,9.00,26.67,...,120.00,18,0,4,0,16.0,16.0,36.0,18.0,3.0
67840,169603,18.0,2.0,1.0,1.0,49.0,216.64,0.11,9.33,24.02,...,117.68,12,0,2,0,8.0,12.0,29.0,22.0,3.0


In [424]:
X_train = train_data.drop(['id',"defects"], axis=1)
X_test = test_data.drop(['id'], axis=1)
y_train = train_data["defects"].apply(int)

In [425]:
# Split the data into training and testing sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [426]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

In [427]:
# Create column transformer (this will help us normalize/preprocess our data)
ct = make_column_transformer(
    (MinMaxScaler(), ["loc", "v(g)", "ev(g)","n","v","l","d","i","e","b","t","lOCode","lOComment", "lOBlank", "locCodeAndComment", "uniq_Op", "uniq_Opnd", "total_Op", "total_Opnd", "branchCount"]), # get all values between 0 and 1
)

In [428]:
# Fit column transformer on the training data only (doing so on test data would result in data leakage)
ct.fit(X_train)

ColumnTransformer(transformers=[('minmaxscaler', MinMaxScaler(),
                                 ['loc', 'v(g)', 'ev(g)', 'n', 'v', 'l', 'd',
                                  'i', 'e', 'b', 't', 'lOCode', 'lOComment',
                                  'lOBlank', 'locCodeAndComment', 'uniq_Op',
                                  'uniq_Opnd', 'total_Op', 'total_Opnd',
                                  'branchCount'])])

In [429]:
X_train_normal = ct.transform(X_train)
X_val_normal = ct.transform(X_val)
X_test_normal = ct.transform(X_test)

In [430]:
# Define the model
model = keras.Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(X_train_normal.shape[1],)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

In [431]:
# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [432]:
# Define the learning rate scheduler
def lr_schedule(epoch):
    if epoch < 5:
        return 1e-3
    else:
        return 1e-3 * (0.9 ** (epoch - 5))

In [433]:
# Create the Early Stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Create the Learning Rate Scheduler callback
lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss',  # Monitor validation loss
    factor=0.5,  # Reduce the learning rate by a factor of 0.5
    patience=3,  # Number of epochs with no improvement before reducing the learning rate
    min_lr=1e-6  # Minimum learning rate
)

In [434]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Train the model
history = model.fit(X_train_normal, y_train, epochs=50, batch_size=32, validation_data=(X_val_normal, y_val))

Epoch 1/50
2545/2545 [==============================] - 14s 5ms/step - loss: 0.4478 - accuracy: 0.8053 - val_loss: 0.4355 - val_accuracy: 0.8120
Epoch 2/50
2545/2545 [==============================] - 12s 5ms/step - loss: 0.4362 - accuracy: 0.8116 - val_loss: 0.4319 - val_accuracy: 0.8131
Epoch 3/50
2545/2545 [==============================] - 11s 4ms/step - loss: 0.4344 - accuracy: 0.8121 - val_loss: 0.4323 - val_accuracy: 0.8114
Epoch 4/50
2545/2545 [==============================] - 12s 5ms/step - loss: 0.4337 - accuracy: 0.8130 - val_loss: 0.4302 - val_accuracy: 0.8152
Epoch 5/50
2545/2545 [==============================] - 11s 4ms/step - loss: 0.4335 - accuracy: 0.8128 - val_loss: 0.4324 - val_accuracy: 0.8138
Epoch 6/50
2545/2545 [==============================] - 13s 5ms/step - loss: 0.4328 - accuracy: 0.8140 - val_loss: 0.4319 - val_accuracy: 0.8144
Epoch 7/50
2545/2545 [==============================] - 12s 5ms/step - loss: 0.4327 - accuracy: 0.8142 - val_loss: 0.4313 - val_ac

In [ ]:
# Load the test data
test_data = pd.read_csv("test.csv")

In [ ]:
# Extract the test IDs
test_ids = test_data['id']

In [ ]:
# Make predictions on the test data
y_pred = model.predict(X_test_normal)

In [ ]:
from sklearn.metrics import roc_auc_score

# Assuming you have your model and validation data prepared as previously mentioned
y_val_pred = model.predict(X_val_normal)

# Calculate the ROC AUC score for the validation data
roc_auc = roc_auc_score(y_val, y_val_pred)

print(f"ROC AUC Score on Validation Data: {roc_auc}")


In [ ]:
# Create a submission file
predictions = pd.DataFrame({'id': test_ids, 'defects': y_pred.flatten()})
predictions.to_csv("submission.csv", index=False)

In [ ]:
import matplotlib.pyplot as plt

# Assuming you have already trained your model and stored the training history in the 'history' variable

# Access accuracy and loss from the training history
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

# Create subplots for accuracy and loss
plt.figure(figsize=(12, 6))

# Accuracy plot
plt.subplot(1, 2, 1)
plt.plot(accuracy, label='Training Accuracy')
plt.plot(val_accuracy, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Loss plot
plt.subplot(1, 2, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Display the plots
plt.tight_layout()
plt.show()
